In [1]:
### This block takes in the full polygon data and cleans it of all uneeded data
## It returns the home plate coordinates as a tuple that reads (longitude, latitude)
## and the polygon coordinates as a list of tuples that read [(longitude, latitude), (longitude, latitude), ...]


import pandas as pd
import re

# Define the extract_lat_lon function
def extract_lat_lon(poly_str):
    coords_str = re.findall(r'(-?\d+\.\d+) (-?\d+\.\d+)', poly_str)
    return [(float(lat), float(lon)) for lon, lat in coords_str]

# Read the CSV data
url = "https://raw.githubusercontent.com/JSmith1826/BB_parks/main/data/360_test/TEMP_plate_fullpoly.csv"
data = pd.read_csv(url)

# Iterate through the rows and apply the function
data['clean_coords'] = data['poly'].apply(extract_lat_lon)

# Print the DataFrame with the new column
# print(data)

## Drop unneeded columns
data = data.drop(['poly'], axis=1)

# rename columns
data = data.rename(columns={'fop_hp': 'home_plate', 'clean_coords': 'polygon'})

data.head()


,field,level,home_plate,polygon
0,Frankfort-Elberta Area HS,high_school,"(44.6344082, -86.2258505)","[(44.6336304, -86.226006), (44.6336295, -86.22..."
1,Ironwood HS,high_school,"(46.457154, -90.1514578)","[(46.4565313, -90.1506035), (46.456498, -90.15..."
2,Oakland A's - mlb,mlb,"(37.7513245, -122.2010782)","[(37.7511479, -122.1999704), (37.7511262, -122..."
3,Citi Field - NY Mets - mlb,mlb,"(40.7565867, -73.8460277)","[(40.7570454, -73.8450072), (40.7568808, -73.8..."
4,Citizens Bank Park - Philadelphia Phillies - mlb,mlb,"(39.9055805, -75.1665943)","[(39.9060907, -75.1656247), (39.9060753, -75.1..."


In [2]:
# ## Check to see if there are any rows with weird or non-existant values

# data[data['home_plate'].isnull()]

# data[data['polygon'].isnull()]

# ## Drop the rows with null values

# data = data.dropna()
# data.info()

# data.dtype()



In [3]:
import math

def distance(home_plate, polygon):
    if isinstance(home_plate, str):
        home_plate = eval(home_plate)
    if isinstance(polygon, str):
        polygon = eval(polygon)
    if len(polygon) < 3:
        raise ValueError('Polygon should be a list of at least three points (x, y)')
    if isinstance(home_plate, (tuple, list)):
        # Home plate is a single coordinate
        if len(home_plate) != 2:
            raise ValueError('Home plate should be a list of two values (longitude, latitude)')
        home_plate = [home_plate]
    else:
        # Home plate is a list of coordinates
        for coord in home_plate:
            if len(coord) != 2:
                raise ValueError('Each home plate coordinate should be a list of two values (longitude, latitude)')
    dist = []
    for home in home_plate:
        for i in polygon:
            # Convert string values to floats
            lon = float(i[0])
            lat = float(i[1])
            # Calculate distance in meters
            dist_m = math.sqrt((home[1] - lat)**2 + (home[0] - lon)**2) * 111319.9 # Distance in meters
            # Convert distance to feet
            dist_ft = dist_m * 3.28084
            dist.append(dist_ft)
    return dist

# Apply the function to the DataFrame
if 'home_plate' in data.columns and 'polygon' in data.columns:
    data['dist'] = data.apply(lambda x: distance(x['home_plate'], x['polygon']), axis=1)
else:
    raise ValueError('DataFrame should have columns named "home_plate" and "polygon"')







In [4]:
data.head()

## Save a csv to check output

data.to_csv('TEMP_plate_fullpoly_dist.csv')

#### THIS LOOKS LIKE IT WORKS !!!!! WORKING ABOVE

### This only measure to each defined coordinate of the polygon and not every 5 degrees around the whole arc like I want it to but it is a start

In [ ]:
### Write a function to measure the distance from the home_plate coordinate to each coordinate in the polygon

## This function takes in the home plate coordinates and the polygon coordinates
## It returns the distance from the home plate to each coordinate in the polygon

import math

def distance(home_plate, polygon):
    dist = []
    for i in polygon:
        # Convert string values to floats
        x = float(i[0])
        y = float(i[1])
        dist.append(math.sqrt((home_plate[0] - x)**2 + (home_plate[1] - y)**2))
    return dist

# Apply the function to the DataFrame
data['dist'] = data.apply(lambda x: distance(x['home_plate'], x['polygon']), axis=1)




In [ ]:
data.head()
df = data

In [ ]:
# print(df)

In [ ]:
import pandas as pd
import math
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
def get_distance_to_edge(home_plate, polygon, angle):
    angle_rad = math.radians(angle)
    step_size = 10  # feet
    max_distance = 1000000  # feet, to avoid infinite loops
    current_distance = 0

    # Get the initial point and move in the direction of the angle
    current_point = Point(home_plate)
    while current_distance < max_distance:
        current_distance += step_size
        new_point = geodesic(feet=current_distance).destination(home_plate[::-1], angle)
        current_point = Point(new_point.longitude, new_point.latitude)
        
        if not polygon.contains(current_point):
            break

    # Calculate the distance in feet
    distance = geodesic(home_plate[::-1], (current_point.y, current_point.x)).feet
    return distance

from numbers import Real

def calculate_distances(row):
    try:
        home_plate = row['home_plate']
        polygon_coords = row['polygon']
        
        # Check if the home_plate and polygon_coords are in the proper format
        if not (isinstance(home_plate, tuple) and len(home_plate) == 2 and
                isinstance(polygon_coords, list) and all(isinstance(coord, tuple) and len(coord) == 2 for coord in polygon_coords) and
                all(isinstance(x, Real) for coord in polygon_coords for x in coord)):
            raise ValueError(f"Invalid format for coordinates: home_plate={home_plate}, polygon_coords={polygon_coords}")

        polygon = Polygon(polygon_coords)
        distances = [get_distance_to_edge(home_plate, polygon, angle) for angle in range(0, 360, 5)]

        return distances

    except Exception as e:
        print(f"Error in row {row.name}: {e}")
        return None

# Assuming df is your DataFrame
df['distances'] = df.apply(calculate_distances, axis=1)


In [ ]:
df.head()